<a href="https://colab.research.google.com/github/Darabu273/IR_engine/blob/master/backend_create_indexes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes



import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 281.3 MB 33 kB/s 
     |████████████████████████████████| 198 kB 41.6 MB/s 
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 100 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-hea

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

**change the following code to the CSV file
with the right corpus**


In [ ]:

pdFromFile = pd.read_csv("/content/smallCourpus.csv")
df_spark = spark.createDataFrame(pdFromFile)


In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

creat a RDD for title, body and anchore


In [ ]:
doc_text_pairs = df_spark.select("text", "id").rdd
doc_title_pairs = df_spark.select("title", "id").rdd
doc_anchor_pairs = df_spark.select("anchor_text", "id").rdd

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(term, id):
  ''' Count the frequency of each word in title\text\anchor (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    term: str
      title\text\anchor of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(term.lower())]
  words_in_dic={}
  word_count=[]
  for tok in tokens:
    if tok not in all_stopwords:
      if tok not in words_in_dic:
        words_in_dic[tok]=1
      else:
        words_in_dic[tok]=words_in_dic[tok]+1

  for tok_count in words_in_dic:

    tpl=(id,words_in_dic[tok_count])
    word_count.append((tok_count,tpl))
  

  return word_count



In [ ]:
#a flatMap() transformation that executes a given function on each element of an RDD, but afterwards flattens the results to create one big "list" instead multiple nested lists
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_anchor = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))


In [ ]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key=lambda x: (x[0])) #To sort by first element of the tuple

In [ ]:
#term , postinglist -> [(doc_id,tf),.....]
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)


In [ ]:
## test reduce_word_count
token, posting_list = postings_text.take(1)[0]
print(posting_list)
print(token)

print('-----')
token, posting_list = postings_title.take(1)[0]
print(posting_list)
print(token)

print('-----')
token, posting_list = postings_anchor.take(1)[0]
print(posting_list)
print(token)

[(3746, 1), (6690, 5), (6788, 1), (7729, 4), (14573, 16), (15858, 1), (31607, 1), (31664, 1), (32593, 6), (43376, 20), (59024, 1), (72852, 2), (72855, 2), (72857, 4), (72858, 1), (72861, 1), (72866, 4), (85533, 1), (149124, 1), (169672, 1), (171581, 14), (240940, 1), (243389, 11), (246185, 1), (255645, 4), (302808, 1), (346029, 2), (412214, 1), (450389, 1), (516570, 1), (565318, 1), (701741, 1), (878666, 1), (903052, 9), (987724, 1), (1060171, 1), (1092923, 1), (1111137, 2), (1533342, 3), (1566357, 47), (1999636, 1), (2103762, 1), (2423824, 1), (2682907, 1), (3188952, 1), (3280233, 2), (3902658, 3), (4055486, 6), (5035602, 3), (5676692, 1), (5774818, 1), (6353598, 1), (7438610, 1), (7971671, 1), (9000355, 1), (9874319, 3), (10062721, 1), (10300434, 1), (11858188, 4), (11891433, 1), (12119521, 1), (13383187, 2), (14759518, 2), (15716827, 1), (16488467, 1), (20089569, 2), (20323494, 2), (22144721, 1), (23388142, 1), (23941708, 3), (25390847, 1), (26275110, 1), (28844729, 1), (36450985, 1

### Document frequency and filteration

In [ ]:
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>50)
postings_filtered_title = postings_title.filter(lambda x: len(x[1])>50)
postings_filtered_anchor = postings_anchor.filter(lambda x: len(x[1])>50)

In [ ]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  token_df = postings.map(lambda x: (x[0],len(x[1])))
  #print(token_df.collect())
  return token_df

In [ ]:
# global statistics
w2df_text = calculate_df(postings_filtered_text)
w2df_title = calculate_df(postings_filtered_title)
w2df_anchor = calculate_df(postings_filtered_anchor)

w2df_dict_text = w2df.collectAsMap()

# test calculate_df types
token, df = w2df_text.take(2)
print(token)
print(df)
print('------')
token, df = w2df_title.take(2)
print(token)
print(df)
print('------')
token, df = w2df_anchor.take(2)
print(token)
print(df)

('walker', 93)
('1982', 218)
------
('yoga', 75)
('chocolate', 74)
------
('row', 2452)
('tom', 208)


#### Inverted index (code from assignment 2)


In [ ]:
# Let's start with a small block size of 30 bytes just to test things out. 
BLOCK_SIZE = 199998

class MultiFileWriter:
    """ Sequential binary writer to multiple files of up to BLOCK_SIZE each. """
    def __init__(self, base_dir, name):
        self._base_dir = Path(base_dir)
        self._name = name
        self._file_gen = (open(self._base_dir / f'{name}_{i:03}.bin', 'wb') 
                          for i in count())
        self._f = next(self._file_gen)
    
    def write(self, b):
      locs = []
      while len(b) > 0:
        pos = self._f.tell()
        remaining = BLOCK_SIZE - pos
        # if the current file is full, close and open a new one.
        if remaining == 0:  
          self._f.close()
          self._f = next(self._file_gen)
          pos, remaining = 0, BLOCK_SIZE
        self._f.write(b[:remaining])
        locs.append((self._f.name, pos))
        b = b[remaining:]
      return locs

    def close(self):
      self._f.close()

class MultiFileReader:
  """ Sequential binary reader of multiple files of up to BLOCK_SIZE each. """
  def __init__(self):
    self._open_files = {}

  def read(self, locs, n_bytes):
    b = []
    for f_name, offset in locs:
      if f_name not in self._open_files:
        self._open_files[f_name] = open(f_name, 'rb')
      f = self._open_files[f_name]
      f.seek(offset)
      n_read = min(n_bytes, BLOCK_SIZE - offset)
      b.append(f.read(n_read))
      n_bytes -= n_read
    return b''.join(b)
  
  def close(self):
    for f in self._open_files.values():
      f.close()

  def __exit__(self, exc_type, exc_value, traceback):
    self.close()
    return False 

In [ ]:
from collections import defaultdict,Counter
import re
import nltk
import pickle
import numpy as np
nltk.download('stopwords')

from nltk.corpus import stopwords
from tqdm import tqdm
import operator
from itertools import islice,count
from contextlib import closing

import json
from io import StringIO
from pathlib import Path
from operator import itemgetter
import pickle
import matplotlib.pyplot as plt

TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this 
                     # many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer

DL = {}  # We're going to update and calculate this after each document. This will be usefull for the calculation of AVGDL (utilized in BM25) 
class InvertedIndex:  
  def __init__(self, docs={}):
    """ Initializes the inverted index and add documents to it (if provided).
    Parameters:
    -----------
      docs: dict mapping doc_id to list of tokens
    """
    # stores document frequency per term
    self.df = Counter()
    # stores total frequency per term
    self.term_total = Counter()
    # stores posting list per term while building the index (internally), 
    # otherwise too big to store in memory.
    self._posting_list = defaultdict(list)
    # mapping a term to posting file locations, which is a list of 
    # (file_name, offset) pairs. Since posting lists are big we are going to
    # write them to disk and just save their location in this list. We are 
    # using the MultiFileWriter helper class to write fixed-size files and store
    # for each term/posting list its list of locations. The offset represents 
    # the number of bytes from the beginning of the file where the posting list
    # starts. 
    self.posting_locs = defaultdict(list)
    
    for doc_id, tokens in docs.items():
      self.add_doc(doc_id, tokens)

  def add_doc(self, doc_id, tokens):
    """ Adds a document to the index with a given `doc_id` and tokens. It counts
        the tf of tokens, then update the index (in memory, no storage 
        side-effects).
    """
    DL[(doc_id)] = DL.get(doc_id,0) + (len(tokens))
    w2cnt = Counter(tokens)
    self.term_total.update(w2cnt)
    max_value = max(w2cnt.items(), key=operator.itemgetter(1))[1]    
    # frequencies = {key: value/max_value for key, value in frequencies.items()}
    for w, cnt in w2cnt.items():        
        self.df[w] = self.df.get(w, 0) + 1                
        self._posting_list[w].append((doc_id, cnt))
        
  def write_index(self, base_dir, name):
    """ Write the in-memory index to disk. Results in the file: 
        (1) `name`.pkl containing the global term stats (e.g. df).
    """
    self._write_globals(base_dir, name)

  def write(self, base_dir, name):
    """ Write the in-memory index to disk and populate the `posting_locs`
        variables with information about file location and offset of posting
        lists. Results in at least two files: 
        (1) posting files `name`XXX.bin containing the posting lists.
        (2) `name`.pkl containing the global term stats (e.g. df).
    """
    #### POSTINGS ####
    self.posting_locs = defaultdict(list)
    with closing(MultiFileWriter(base_dir, name)) as writer:
      # iterate over posting lists in lexicographic order
      for w in sorted(self._posting_list.keys()):
        self._write_a_posting_list(w, writer, sort=True)
    #### GLOBAL DICTIONARIES ####
    self._write_globals(base_dir, name)

  def _write_globals(self, base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'wb') as f:
      pickle.dump(self, f)

  def write_a_posting_list(b_w_pl):
    ''' Takes a (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...]) 
    and writes it out to disk as files named {bucket_id}_XXX.bin under the 
    current directory. Returns a posting locations dictionary that maps each 
    word to the list of files and offsets that contain its posting list.
    Parameters:
    -----------
      b_w_pl: tuple
        Containing a bucket id and all (word, posting list) pairs in that bucket
        (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...])
    Return:
      posting_locs: dict
        Posting locations for each of the words written out in this bucket.
    '''
    posting_locs = defaultdict(list)
    bucket, list_w_pl = b_w_pl

    with closing(MultiFileWriter('.', bucket)) as writer:
      for w, pl in list_w_pl: 
        # convert to bytes
        b = b''.join([(doc_id << 16 | (tf & TF_MASK)).to_bytes(TUPLE_SIZE, 'big')
                      for doc_id, tf in pl])
        # write to file(s)
        locs = writer.write(b)
      # save file locations to index
        posting_locs[w].extend(locs)
    return posting_locs
  # def _write_a_posting_list(self, w, writer, sort=False):
  #   # sort the posting list by doc_id
  #   pl = self._posting_list[w]
  #   if sort:
  #     pl = sorted(pl, key=itemgetter(0))
  #   # convert to bytes    
  #   b = b''.join([(int(doc_id) << 16 | (tf & TF_MASK)).to_bytes(TUPLE_SIZE, 'big')
  #                 for doc_id, tf in pl])
  #   # write to file(s)
  #   locs = writer.write(b)
  #   # save file locations to index
  #   self.posting_locs[w].extend(locs) 

  def __getstate__(self):
    """ Modify how the object is pickled by removing the internal posting lists
        from the object's state dictionary. 
    """
    state = self.__dict__.copy()
    del state['_posting_list']
    return state

  @staticmethod
  def read_index(base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'rb') as f:
      return pickle.load(f)

  @staticmethod
  def delete_index(base_dir, name):
    path_globals = Path(base_dir) / f'{name}.pkl'
    path_globals.unlink()
    for p in Path(base_dir).rglob(f'{name}_*.bin'):
      p.unlink()
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!




**partitions the posting list, writes out each bucket, and returns information about the location on disk of each posting list.**

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  bucket_num = postings.map(lambda x: (token2bucket_id(x[0]),x))
  dicAll_union = bucket_num.groupByKey()
  return dicAll_union.map(lambda b_w_pl:InvertedIndex.write_a_posting_list(b_w_pl))


In [ ]:
posting_locs_list_text = partition_postings_and_write(postings_filtered_text).collect()
posting_locs_list_title = partition_postings_and_write(postings_filtered_title).collect()
posting_locs_list_anchor = partition_postings_and_write(postings_filtered_anchor).collect()


Putting everything together (posting locations, df) - create 3 inverted index for text, title and anchor 






In [ ]:
#text
# merge the posting locations into a single dict
super_posting_locs_text = defaultdict(list)
for posting_loc in posting_locs_list_text:
  for k, v in posting_loc.items():
    super_posting_locs_text[k].extend(v)
# Create inverted index instance
inverted_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs_text
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_dict_text
# write the global stats out
inverted_text.write_index('.', 'index')

In [ ]:
#title
# merge the posting locations into a single dict
super_posting_locs_title = defaultdict(list)
for posting_loc in posting_locs_list_title:
  for k, v in posting_loc.items():
    super_posting_locs_title[k].extend(v)
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict_text
# write the global stats out
inverted_title.write_index('.', 'index')

In [ ]:
#anchor
# merge the posting locations into a single dict
super_posting_locs_anchor = defaultdict(list)
for posting_loc in posting_locs_list_anchor:
  for k, v in posting_loc.items():
    super_posting_locs_anchor[k].extend(v)
# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_anchor
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_dict_text
# write the global stats out
inverted_anchor.write_index('.', 'index')